In [274]:
import numpy as np
import pandas as pd
import ast

In [275]:
movie=pd.read_csv('tmdb_5000_movies.csv')
credit=pd.read_csv('tmdb_5000_credits.csv')

In [276]:
movie=pd.merge(movie,credit,on='title')

In [277]:
movie=movie[['id','title','genres','cast','crew','overview','keywords']]

In [278]:
movie['overview'] = movie['overview'].apply(lambda x: x.split() if isinstance(x, str) else x)


In [279]:
def convert(obj):
    li=[]
    for i in ast.literal_eval(obj):
        li.append(i['name'])
    return li

In [280]:

movie['genres']=movie['genres'].apply(convert)

In [281]:
movie['keywords']=movie['keywords'].apply(convert)

In [282]:
def convert2(obj):
    li=[]
    count=0;
    
    for i in ast.literal_eval(obj):
        if count<=3:
            li.append(i['name'])
            count+=1
        else:
            break
    return li

In [283]:
movie['cast']=movie['cast'].apply(convert2)

In [284]:
def convert3(obj):
    li = []
    data = ast.literal_eval(obj)
    for i in data:
        if i['job'] == 'Director':
            li.append(i['name'])
            break
        
    return li

In [285]:
movie['crew']=movie['crew'].apply(convert3)

In [286]:
movie['cast']=movie['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movie['crew']=movie['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movie['genres']=movie['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movie['keywords']=movie['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [287]:
movie['tags']=movie['genres']+movie['cast']+movie['crew']+movie['overview']+movie['keywords']
movie=movie[['id','title','tags']]

In [288]:
movie['tags'] = movie['tags'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
movie['tags'] = movie['tags'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [289]:
movie.dropna(inplace=True)

In [290]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [291]:
vectors=cv.fit_transform(movie['tags']).toarray()

In [292]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [293]:
def origin(txt):
    li=[]
    txt=txt.split()
    for i in txt:
        li.append(ps.stem(i))
    return " ".join(li)

In [294]:
movie['tags']=movie['tags'].apply(origin)

In [295]:
from sklearn.metrics.pairwise import cosine_similarity
simlarity=cosine_similarity(vectors)

In [301]:
def recom(mov):
    mov_ind=movie[movie['title']==mov].index[0]
    distance=simlarity[mov_ind]
    li=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    for i in li:
        print(movie.iloc[i[0]].title)

In [312]:
movie[movie['title']=='Avatar'].index[0]

0

In [302]:
recom('Batman ')

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman


In [303]:
recom('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman


In [304]:
recom('Avatar')

Titan A.E.
Independence Day
Small Soldiers
Aliens vs Predator: Requiem
Ender's Game


In [305]:
recom('Spectre')

Quantum of Solace
Never Say Never Again
Skyfall
From Russia with Love
Thunderball
